# BIODATA
## Nama: Feryadi Yulius
## Kelas: Generative AI with LLM

# NAVASENA: Sistem Rekomendasi Outfit

Notebook ini mendemonstrasikan pembuatan sistem rekomendasi outfit. Sistem ini akan memanfaatkan dataset rekomendasi outfit, membuat vector store untuk pencarian cepat, dan menggunakan Large Language Model (LLM) dari NVIDIA (melalui API Endpoints) untuk menghasilkan rekomendasi berdasarkan input pengguna dan konteks yang diambil dari dataset.

**Tujuan Utama:**
1.  Mempersiapkan environment dengan dependensi yang diperlukan.
2.  Memuat dan melakukan pra-pemrosesan dataset rekomendasi outfit.
3.  Membangun vector store menggunakan FAISS dan embeddings dari Sentence Transformers.
4.  Mengkonfigurasi dan menggunakan LLM dari NVIDIA untuk menghasilkan rekomendasi.
5.  Mengintegrasikan retriever (dari vector store) dengan LLM untuk membentuk RAG pipeline.

**Penting:**
* Pastikan Anda menjalankan notebook ini di environment yang memiliki GPU untuk akselerasi, terutama untuk embedding dan model LLM jika memungkinkan. Colab dengan GPU direkomendasikan.
* **ANDA MEMBUTUHKAN NVIDIA API KEY** untuk mengakses model LLM. Simpan API key Anda dengan aman (misalnya, menggunakan Colab Secrets). **JANGAN HARDCODE API KEY LANGSUNG DI DALAM KODE SECARA PERMANEN, TERUTAMA JIKA AKAN DI-SHARE.**

# 0. ENVIRONMENT PREPARATION

### 0.1. Penyesuaian Versi NumPy
Beberapa dependensi, terutama yang berkaitan dengan NeMo atau PyTorch, mungkin memerlukan versi NumPy tertentu untuk kompatibilitas. Sel ini memastikan NumPy versi `1.26.4` terinstal. Jika versi lain terdeteksi atau NumPy belum terinstal, versi yang diinginkan akan diinstal, dan runtime Colab akan di-restart secara otomatis (`os._exit(00)`) untuk menerapkan perubahan.

In [ ]:
import importlib.metadata

desired_version = "1.26.4"

try:
    installed_version = importlib.metadata.version("numpy")
    if installed_version == desired_version:
        print(f"NumPy {desired_version} is already installed.")
    else:
        print(f"Installing NumPy {desired_version} (current: {installed_version})...")
        !pip install numpy=={desired_version} --prefer-binary
        import os
        os._exit(00)  # Restart runtime for changes to take effect
except importlib.metadata.PackageNotFoundError:
    print(f"NumPy is not installed. Installing {desired_version}...")
    !pip install numpy=={desired_version} --prefer-binary
    import os
    os._exit(00)

NumPy 1.26.4 is already installed.


### 0.2. Instalasi `uv`
`uv` adalah package installer alternatif untuk Python yang seringkali lebih cepat daripada `pip`. Kita akan menggunakannya untuk beberapa instalasi berikutnya.

In [ ]:
!pip install uv

### 0.3. Instalasi Dependensi Tambahan dan NeMo Toolkit

Sel ini menginstal beberapa paket sistem dan Python yang diperlukan:
* **Paket Sistem (via `apt-get`):**
    * `sox`: Utilitas pemrosesan audio.
    * `libsndfile1`: Library untuk membaca/menulis file audio.
    * `ffmpeg`: Framework multimedia serbaguna.
* **Paket Python (via `uv pip`):**
    * `wget`: Untuk mengunduh file.
    * `text-unidecode`: Untuk transliterasi teks Unicode ke ASCII.
* **NVIDIA NeMo Toolkit:**
    * NeMo adalah toolkit open-source untuk membangun dan melatih model AI percakapan. Di sini, kita menginstalnya langsung dari repositori GitHub (branch `main`) dengan semua dependensi opsional (`[all]`). Meskipun NeMo mungkin tidak digunakan secara langsung di versi final notebook ini (karena kita beralih ke LangChain dan NVIDIA NIM), sel ini adalah bagian dari setup awal yang mungkin relevan jika ada eksplorasi NeMo lebih lanjut.

**Catatan untuk Pengguna Colab:**
Pastikan Anda telah menghubungkan runtime ke GPU (Runtime -> Change runtime type -> GPU) sebelum menjalankan sel ini.

In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!apt-get install sox libsndfile1 ffmpeg
!uv pip install wget
!uv pip install text-unidecode

### Install NeMo
# BRANCH = 'r2.0.0rc0'
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
sox is already the newest version (14.4.2+git20190427-2+deb11u2ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Using Python 3.11.12 environment at: /usr
Audited 1 package in 76ms
Using Python 3.11.12 environment at: /usr
Audited 1 package in 89ms
DEPRECATION: git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[all] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-kp54j97x/nemo-toolkit_6133204fa04a45b59a3d1874f7a5e735
  Running command git clone --

### 0.4. Verifikasi Versi NeMo dan NumPy
Setelah instalasi, kita akan memeriksa versi NeMo dan NumPy yang terinstal untuk memastikan semuanya berjalan sesuai harapan.

In [ ]:
import nemo
nemo.__version__

'2.4.0rc0'

In [ ]:
import numpy as np

print(np.__version__)

1.26.4


### 0.5. Instalasi LangChain dan Paket Terkait
Selanjutnya, kita menginstal pustaka inti untuk membangun aplikasi LLM:
* `langchain`: Framework untuk mengembangkan aplikasi yang ditenagai oleh LLM.
* `langgraph`: Ekstensi LangChain untuk membangun agen LLM yang stateful dan multi-aktor.
* `langchain_nvidia_ai_endpoints`: Integrasi LangChain dengan NVIDIA AI Foundation Endpoints.

*Catatan: Sel ini dijalankan dua kali, sekali dengan `pip` dan sekali dengan `uv pip`. Ini mungkin redundan; salah satu saja (misalnya, `uv pip`) seharusnya cukup.*

In [ ]:
# Install langchain dan langchain-nvidia
!uv pip install -q langchain langgraph langchain_nvidia_ai_endpoints

In [ ]:
!uv pip install -U langchain-community

Using Python 3.11.12 environment at: /usr
Resolved 46 packages in 850ms
⠙ Preparing packages... (0/7)
⠙ Preparing packages... (0/7)
⠙ Preparing packages... (0/7)
⠙ Preparing packages... (0/7)
⠙ Preparing packages... (0/7)
⠙ Preparing packages... (0/7)
⠙ Preparing packages... (0/7)
⠙ Preparing packages... (0/7)
⠙ Preparing packages... (0/7)
urllib3              ------------------------------ 91.19 KiB/125.66 KiB
⠙ Preparing packages... (0/7)
urllib3              ------------------------------ 91.19 KiB/125.66 KiB
⠙ Preparing packages... (0/7)
urllib3              ------------------------------ 91.19 KiB/125.66 KiB
⠙ Preparing packages... (0/7)
urllib3              ------------------------------ 91.19 KiB/125.66 KiB
langsmith            ------------------------------     0 B/353.51 KiB
⠙ Preparing packages... (0/7)
urllib3              ------------------------------ 91.19 KiB/125.66 KiB
langsmith            ------------------------------     0 B/353.51 KiB
⠙ Preparing packages... (0/7)
u

### 0.6. Impor Pustaka yang Dibutuhkan
Mengimpor semua modul Python yang akan digunakan sepanjang notebook.

*Catatan: Output sel ini mungkin menunjukkan `LangChainDeprecationWarning` terkait `langchain_core.pydantic_v1`. Ini adalah pengingat untuk memperbarui ke Pydantic v2 jika memungkinkan di masa mendatang, tetapi untuk saat ini, kode masih akan berfungsi.*

In [ ]:
import os
import pandas as pd
import json
import requests
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import create_react_agent
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from IPython.display import Image, display

# 1. DATA LOADING

### 1.1. Memuat Dataset dari Hugging Face Hub
Dataset rekomendasi outfit (`new_data.csv`) dimuat langsung dari Hugging Face Hub menggunakan path `hf://`. Pandas akan menangani pengunduhan dan parsing file CSV ke dalam DataFrame.

In [ ]:
df = pd.read_csv("hf://datasets/formido/outfit_recomendation/new_data.csv")

### 1.2. Inspeksi Struktur Dataset
Menampilkan informasi ringkas tentang DataFrame, termasuk tipe data setiap kolom dan jumlah nilai non-null. Ini membantu memahami struktur data awal.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3193 entries, 0 to 3192
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  3186 non-null   object
 1   input        3193 non-null   object
 2   output       3193 non-null   object
dtypes: object(3)
memory usage: 75.0+ KB


### 1.3. Pra-pemrosesan dan Konversi Data ke Format JSONL

Untuk digunakan dalam sistem, data perlu diproses:
1.  **Kombinasi Prompt:** Kolom `instruction` dan `input` dari dataset digabungkan menjadi satu string prompt tunggal, dipisahkan oleh token `[SEP]`. Fungsi `combine_prompt` menangani nilai yang mungkin hilang (NaN).
2.  **Format JSONL:** Setiap baris data diubah menjadi objek JSON dengan dua kunci:
    * `text`: Berisi prompt gabungan.
    * `response`: Berisi kolom `output` dari dataset.
3.  **Penyimpanan:** Data yang telah diproses disimpan dalam format JSON Lines (`.jsonl`), di mana setiap baris adalah objek JSON yang valid. Format ini umum digunakan untuk dataset LLM. Entri hanya disimpan jika `prompt` dan `output` keduanya valid (tidak kosong).

### 1.3. Pra-pemrosesan dan Konversi Data ke Format JSONL

Untuk digunakan dalam sistem RAG (atau potensi fine-tuning LLM), data perlu diproses:
1.  **Kombinasi Prompt:** Kolom `instruction` dan `input` dari dataset digabungkan menjadi satu string prompt tunggal, dipisahkan oleh token `[SEP]`. Fungsi `combine_prompt` menangani nilai yang mungkin hilang (NaN).
2.  **Format JSONL:** Setiap baris data diubah menjadi objek JSON dengan dua kunci:
    * `text`: Berisi prompt gabungan.
    * `response`: Berisi kolom `output` dari dataset.
3.  **Penyimpanan:** Data yang telah diproses disimpan dalam format JSON Lines (`.jsonl`), di mana setiap baris adalah objek JSON yang valid. Format ini umum digunakan untuk dataset LLM. Entri hanya disimpan jika `prompt` dan `output` keduanya valid (tidak kosong).

In [ ]:
def combine_prompt(row):
    instr = str(row.get('instruction', '')).strip() if pd.notna(row.get('instruction')) else ''
    inp = str(row.get('input', '')).strip() if pd.notna(row.get('input')) else ''
    return f"{instr} [SEP] {inp}".strip()

jsonl_data = []

for _, row in df.iterrows():
    prompt = combine_prompt(row)
    output = str(row.get("output", "")).strip() if pd.notna(row.get("output")) else ""

    if prompt and output:
        jsonl_data.append({
            "text": prompt,
            "response": output
        })

output_path = "outfit_recommendation_dataset.jsonl"

with open(output_path, 'w', encoding='utf-8') as f:
    for entry in jsonl_data:
        f.write(json.dumps(entry, ensure_ascii=False) + '\n')

print(f"✅ Berhasil dikonversi ke: {output_path}, total entri: {len(jsonl_data)}")

✅ Berhasil dikonversi ke: outfit_recommendation_dataset.jsonl, total entri: 3193


# 2. MEMBANGUN SISTEM

Bagian ini akan fokus pada pembangunan komponen inti dari sistem RAG:
1.  Memuat teks yang telah diproses (prompt dari dataset).
2.  Menginisialisasi model embedding untuk mengubah teks menjadi vektor.
3.  Membuat vector store (FAISS) untuk menyimpan dan mencari vektor teks secara efisien.
4.  Mengkonfigurasi LLM dari NVIDIA.
5.  Menggabungkan retriever dan LLM menjadi sebuah chain.

### 2.1. Memuat Teks untuk Vector Store
Teks (prompt gabungan) yang sebelumnya disimpan dalam file `outfit_recommendation_dataset.jsonl` dimuat ke dalam list. Teks-teks ini akan digunakan untuk membangun vector store.

In [ ]:
jsonl_path = "outfit_recommendation_dataset.jsonl"

texts = []
for line in open(jsonl_path, 'r', encoding='utf-8'):
    obj = json.loads(line)
    texts.append(obj["text"])

### 2.2. Inisialisasi Model Embeddings dan Membuat Vector Store

1.  **Model Embeddings:**
    * Kita menggunakan model `sentence-transformers/all-MiniLM-L6-v2` dari Hugging Face. Model ini efisien dan memberikan embedding berkualitas baik untuk teks.
    * `HuggingFaceEmbeddings` dari LangChain digunakan untuk memuat model ini, dengan `model_kwargs={"device": "cuda"}` untuk memastikan pemrosesan berjalan di GPU jika tersedia.
    * *Catatan Deprekasi:* `HuggingFaceEmbeddings` dari `langchain.embeddings` telah dipindahkan. Sebaiknya gunakan versi dari `langchain_community.embeddings` atau `langchain_huggingface`.
2.  **Vector Store (FAISS):**
    * FAISS (Facebook AI Similarity Search) adalah library untuk pencarian kemiripan yang efisien dan clustering vektor padat.
    * `FAISS.from_texts()` digunakan untuk membuat vector store langsung dari list `texts` dan model `embeddings` yang telah diinisialisasi. Setiap teks akan diubah menjadi vektor embedding dan diindeks oleh FAISS.
    * *Catatan Deprekasi:* `FAISS` dari `langchain.vectorstores` telah dipindahkan. Sebaiknya gunakan versi dari `langchain_community.vectorstores`.

Output dari sel ini akan menampilkan progress bar saat model embedding diunduh (jika belum ada di cache) dan saat teks-teks di-embed dan diindeks.

In [ ]:
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs={"device": "cuda"})

vectorstore = FAISS.from_texts(texts, embeddings)

### 2.3. Konfigurasi LLM dari NVIDIA AI Endpoints

Di sini, kita akan mengkonfigurasi Large Language Model (LLM) yang akan digunakan untuk menghasilkan jawaban. Kita menggunakan `ChatNVIDIA` dari `langchain_nvidia_ai_endpoints`.

* **`base_url`**: URL endpoint untuk NVIDIA API.
* **`api_key`**: Kunci API Anda untuk NVIDIA. **PENTING:** Ganti nilai placeholder di bawah ini dengan API key Anda yang sebenarnya, atau lebih baik lagi, gunakan Colab Secrets seperti yang dijelaskan sebelumnya.
* **`model`**: Nama model yang akan digunakan. Anda dapat menemukan daftar model yang tersedia di [NVIDIA AI Foundation Models Dashboard](https://build.nvidia.com/explore/discover). Model yang dipilih di sini adalah `nvidia/llama-3.1-nemotron-nano-vl-8b-v1`.
* **`temperature`**: Mengontrol kreativitas output. Nilai yang lebih rendah (misalnya, 0.3) menghasilkan output yang lebih deterministik dan fokus.



In [ ]:
from google.colab import userdata

In [ ]:
base_url = "https://integrate.api.nvidia.com/v1"
api_key = userdata.get('NVIDIA_API_KEY')

model = "nvidia/llama-3.1-nemotron-nano-vl-8b-v1"

llm = ChatNVIDIA(
    base_url=base_url,
    model=model,
    temperature=0.3,
    api_key=api_key
)

/usr/local/lib/python3.11/dist-packages/langchain_nvidia_ai_endpoints/_common.py:217: UserWarning: Found nvidia/llama-3.1-nemotron-nano-vl-8b-v1 in available_models, but type is unknown and inference may fail.
  warnings.warn(


### 2.4. (Opsional) Verifikasi API Key dan Daftar Model
Sel ini melakukan panggilan langsung ke endpoint NVIDIA API untuk mengambil daftar model yang tersedia. Ini adalah cara yang baik untuk memverifikasi bahwa API key Anda valid dan dapat mengakses layanan. Jika `response.status_code` adalah `200`, koneksi berhasil. Output juga akan menampilkan daftar model dalam format JSON.

In [ ]:
headers = {"Authorization": f"Bearer {api_key}"}
response = requests.get("https://integrate.api.nvidia.com/v1/models", headers=headers)
print(response.status_code)
print(response.text)

200
{"object":"list","data":[{"id":"01-ai/yi-large","object":"model","created":735790403,"owned_by":"01-ai"},{"id":"abacusai/dracarys-llama-3.1-70b-instruct","object":"model","created":735790403,"owned_by":"abacusai"},{"id":"adept/fuyu-8b","object":"model","created":735790403,"owned_by":"adept"},{"id":"ai21labs/jamba-1.5-large-instruct","object":"model","created":735790403,"owned_by":"ai21labs"},{"id":"ai21labs/jamba-1.5-mini-instruct","object":"model","created":735790403,"owned_by":"ai21labs"},{"id":"aisingapore/sea-lion-7b-instruct","object":"model","created":735790403,"owned_by":"aisingapore"},{"id":"baai/bge-m3","object":"model","created":735790403,"owned_by":"baai"},{"id":"baichuan-inc/baichuan2-13b-chat","object":"model","created":735790403,"owned_by":"baichuan-inc"},{"id":"bigcode/starcoder2-15b","object":"model","created":735790403,"owned_by":"bigcode"},{"id":"bigcode/starcoder2-7b","object":"model","created":735790403,"owned_by":"bigcode"},{"id":"databricks/dbrx-instruct","obj

### 2.5. Menggabungkan Retriever dan LLM: RAG Chain

Setelah vector store dan LLM dikonfigurasi, kita menggabungkannya menjadi sebuah RAG (Retrieval Augmented Generation) chain:
1.  **Retriever:**
    * `vectorstore.as_retriever()` mengubah vector store FAISS kita menjadi objek retriever LangChain.
    * `search_type="similarity"`: Menginstruksikan retriever untuk mencari dokumen berdasarkan kemiripan kosinus (cosine similarity).
    * `search_kwargs={"k": 3}`: Retriever akan mengambil 3 dokumen teratas yang paling mirip dengan query pengguna.
2.  **RAG Chain (`RetrievalQA`):**
    * `RetrievalQA` adalah chain yang mengkombinasikan retriever dengan LLM.
    * `llm=llm`: Menggunakan LLM NVIDIA yang telah dikonfigurasi.
    * `chain_type="stuff"`: Salah satu metode untuk memasukkan dokumen yang diambil ke dalam prompt LLM. Metode "stuff" akan menggabungkan semua dokumen yang diambil ke dalam satu prompt. Cocok jika total panjang dokumen tidak terlalu besar.
    * `retriever=retriever`: Menggunakan retriever yang baru saja dibuat.

Chain ini akan:
1.  Menerima query pengguna.
2.  Menggunakan retriever untuk menemukan dokumen (potongan teks dari dataset kita) yang relevan.
3.  Memasukkan query pengguna dan dokumen yang relevan ke dalam LLM.
4.  LLM kemudian menghasilkan jawaban berdasarkan informasi tersebut.

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

# 3. MENGGUNAKAN SISTEM REKOMENDASI OUTFIT

Sekarang sistem RAG telah siap, kita dapat mengajukan pertanyaan atau permintaan rekomendasi outfit.

### 3.1. Contoh Query
Kita akan mencoba query: "Berikan rekomendasi outfit untuk acara formal dengan cuaca dingin".
Chain `qa_chain` akan memproses query ini dan menghasilkan rekomendasi.

In [ ]:
print("Welcome to the Interactive Outfit Recommendation System!")
print("Type 'exit' at any time to quit the program.")
print("-" * 50)

while True:
    user_query = input("\nYou: ")
    if user_query.lower() == 'exit':
        print("\nThank you for using the recommendation system. Goodbye!")
        break
    if not user_query.strip():
        print("Please enter your question.")
        continue

    print("\nModel is thinking...")
    try:
        result = qa_chain.run(user_query)
        print("\nModel's Answer:")
        print(result)
    except Exception as e:
        print(f"Sorry, an error occurred while processing your request: {e}")
    print("-" * 50)

Welcome to the Interactive Outfit Recommendation System!
Type 'exit' at any time to quit the program.
--------------------------------------------------

You: halo

Model is thinking...

Model's Answer:
Hello!
--------------------------------------------------

You: i am ready to hiking with my friend

Model is thinking...

Model's Answer:
It sounds like you're looking for advice on what to wear for a hiking trip in a national park. 

For a comfortable and practical outfit, consider wearing moisture-wicking, breathable clothing and sturdy hiking boots. You may also want to bring layers for changing weather conditions. Don't forget to pack sunscreen, a hat, and sunglasses for protection from the sun. 

Additionally, it's a good idea to bring enough water, snacks, and a first-aid kit for your hike. Make sure to check the weather forecast and trail conditions before you go and let someone know your hiking plans. 

Have a safe and enjoyable hike with your friend!
--------------------------

# 4. Simpan FAISS_INDEX_PATH


In [ ]:
FAISS_INDEX_PATH = "faiss_outfit_recommendation_index" # Pastikan nama ini sama
vectorstore.save_local(FAISS_INDEX_PATH)
print(f"Vector store saved to {FAISS_INDEX_PATH}")

Vector store saved to faiss_outfit_recommendation_index


# 5. Prompt Engineriing

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_template = """You are a friendly and helpful expert fashion stylist. Based on the following preferences and event information, provide detailed and stylish outfit recommendations.
If the information provided is insufficient, ask for relevant additional details.

Context from data:
{context}

User's Question: {question}

Outfit Recommendation (explain each item and why it's suitable):
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# Saat membuat chain:
qa_chain = RetrievalQA.from_chain_type(
llm=llm,
chain_type="stuff",
retriever=retriever,
chain_type_kwargs={"prompt": PROMPT}
)

In [ ]:
print("Welcome to the Interactive Outfit Recommendation System!")
print("Type 'exit' at any time to quit the program.")
print("-" * 50)

while True:
    user_query = input("\nYou: ")
    if user_query.lower() == 'exit':
        print("\nThank you for using the recommendation system. Goodbye!")
        break
    if not user_query.strip():
        print("Please enter your question.")
        continue

    print("\nModel is thinking...")
    try:
        result = qa_chain.run(user_query)
        print("\nModel's Answer:")
        print(result)
    except Exception as e:
        print(f"Sorry, an error occurred while processing your request: {e}")
    print("-" * 50)

Welcome to the Interactive Outfit Recommendation System!
Type 'exit' at any time to quit the program.
--------------------------------------------------

You: i am ready to hiking with my friend

Model is thinking...

Model's Answer:
Sure, I'd be happy to help you with your outfit recommendations for hiking in a national park. Here's a suggestion for each of you:

For the broad-chested, tall man who loves classic, refined clothing:

* A pair of high-quality, breathable hiking pants in a neutral color such as khaki or navy blue. These should be made of a durable, moisture-wicking fabric that will keep you comfortable during your hike.
* A lightweight, moisture-wicking hiking shirt in a color that complements your pants. Consider a long-sleeved shirt for sun protection, or a short-sleeved shirt for warmer weather.
* A pair of sturdy hiking boots that are appropriate for the terrain you'll be hiking on. Look for boots with good ankle support and a comfortable, cushioned insole.
* A wide-b